In [ ]:
import pandas as pd
from sklearn import feature_selection, tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

from itertools import product, combinations
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Get Data for Training...

data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
# Get Feature Data...

df_path = "./../FFChallenge_v2/background.dta"

df_features = None
with open(df_path, "r") as f:
    df_features = pd.read_stata(f)
    print df_features.head()
    
df_train = df_features.set_index('challengeID')
df_train_na = df_train.replace('NA', np.NaN)
df_train_na.cf4fint = pd.to_datetime(df_train_na.cf4fint)
df_train_na_cols = df_train_na.columns[df_train_na.dtypes == 'object']
df_train_na[df_train_na_cols] = df_train_na[df_train_na_cols].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))

df_train_no_obj = df_train_na[df_train_na.columns[df_train_na.dtypes != 'object']]
final_cols = df_train_no_obj.columns[~ df_train_no_obj.isnull().all()]
df_final = pd.DataFrame(df_train_no_obj[final_cols])
print df_final.shape

# Find number of unique values in each column. If unique == 1, then remove from final data frame.
n = df_final.apply(lambda x: len(x.unique()))
df_final = pd.DataFrame(df_final[df_final.columns[n>1]])
print df_final.shape

In [ ]:
cols_interest = [
    'm2h19e',
    'm2h19d',
    'm2h19f',
    'm2h19g',
    'm2h19h',
#    'f2h17e',
 #   'f2h17d',
#    'f2h17f',
#    'f2h17g',
#    'f2h17h',
    'm3i23c',
    'm3i23b',
    'm3i23d',
    'f3i23c',
    'f3i23b',
    'f3i23d',
    'm4i23e',
    'm4i23d',
    'm4i23f',
    'm4i23g',
    'm4i23h',
    'm5f23e',
    'm5f23d',
    'm5f23f',
    'm5f23g',
    'm5f23h',
    'n5g1d',
    'n5g1e',
    'n5g1c',
]

df = pd.DataFrame(df_final[cols_interest])


In [ ]:
evict = df_test.eviction.dropna()

In [ ]:
train = df.loc[evict.index]

In [ ]:
X = train.apply(lambda x: x.cat.codes)
X['sum'] = X.sum(axis=1)
Y = evict

In [ ]:
def checkNegative(x):
    return isinstance(x, basestring) and x.startswith('-')

In [ ]:
chi2, pval = feature_selection.chi2(X,Y)
feat_indx = X.columns[~np.isnan(chi2)]
chi2_nona = chi2[~np.isnan(chi2)]
pval_nona = pval[~np.isnan(chi2)]
n_unique = X.apply(lambda x: len(x.unique()))
n_nan = X.apply(lambda x: x.isnull().sum())
n_other = train.applymap(checkNegative).sum()

feat_rank = pd.DataFrame({
    'chi2': chi2_nona, 
    'pval': pval_nona,
    'unqe': n_unique,
    'n_nan': n_nan,
    'other': n_other,
}, index = feat_indx)

In [ ]:
ordered = feat_rank.sort_values('pval')
print ordered

In [ ]:
cols = ordered.index[0:]
cutoff = 500
#clf = MultinomialNB()
#clf = tree.DecisionTreeClassifier()
clf = SVC(probability=True)
clf.fit(X[cols].iloc[:cutoff], Y[:cutoff])

In [ ]:
out = clf.predict(X[cols].iloc[cutoff:])
out_p = clf.predict_proba(X[cols].iloc[cutoff:])
plt.hist(out)
print metrics.classification_report(Y[cutoff:], out)
print metrics.confusion_matrix(Y[cutoff:], out)
print metrics.brier_score_loss(Y[cutoff:], out)
print metrics.brier_score_loss(Y[cutoff:], out_p[:,1])

In [ ]:
out_p

In [ ]:
X['m5f23e'][Y==0].hist(normed=True, alpha=0.5)
X['m5f23e'][Y==1].hist(normed=True, alpha=0.5)

In [ ]:
df['m5f23e']